<a href="https://colab.research.google.com/github/Ratapakorn/chatbot-profile/blob/main/nb/Gemma3_(1B)-GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth your local device, follow [our guide](https://docs.unsloth.ai/get-started/install-and-update). This notebook is licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News


Unsloth's [Docker image](https://hub.docker.com/r/unsloth/unsloth) is here! Start training with no setup & environment issues. [Read our Guide](https://docs.unsloth.ai/new/how-to-train-llms-with-unsloth-and-docker).

[gpt-oss RL](https://docs.unsloth.ai/new/gpt-oss-reinforcement-learning) is now supported with the fastest inference & lowest VRAM. Try our [new notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/gpt-oss-(20B)-GRPO.ipynb) which creates kernels!

Introducing [Vision](https://docs.unsloth.ai/new/vision-reinforcement-learning-vlm-rl) and [Standby](https://docs.unsloth.ai/basics/memory-efficient-rl) for RL! Train Qwen, Gemma etc. VLMs with GSPO - even faster with less VRAM.

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [20]:
%%capture
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1" # [NEW] Extra 30% context lengths!
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install or uv pip install
    !pip install unsloth vllm
else:
    pass # For Colab / Kaggle, we need extra instructions hidden below \/

In [3]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2

### Unsloth

Load up `Gemma 3 1B Instruct`, and set parameters

In [4]:
from unsloth import FastModel
import torch
max_seq_length = 1024

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 11-05 03:29:34 [__init__.py:244] Automatically detected platform cuda.
ERROR 11-05 03:29:37 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.1: Fast Gemma3 patching. Transformers: 4.56.2. vLLM: 0.9.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.
Unsloth: QLoRA and full finetuning al

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

We now add LoRA adapters so we only need to update a small amount of parameters!

In [5]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


### Data Prep
<a name="Data"></a>

We're using OpenAI's famous GSM8K dataset!

In [78]:
# from datasets import load_dataset
# dataset = load_dataset("openai/gsm8k", "main", split = "train")
# dataset

from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files="labels.jsonl",  # one JSON object per line
    split="train"
)
print(dataset)

Dataset({
    features: ['uid', 'user', 'response', 'label', 'rationale', 'ts', 'dataset', 'split', 'row_index'],
    num_rows: 100
})


Let's look at the first row:

In [79]:
str(dataset[0]["user"])

"{'event_id': '8e785e09-5213-46b1-a6eb-b7e40998905b', 'timestamp': datetime.datetime(2025, 5, 28, 23, 46, 49), 'event_type': 'endpoint', 'source': 'Microsoft Sentinel v1.0.0', 'severity': 'critical', 'raw_log': 'CEF:0|Microsoft Sentinel v1.0.0|SIEM|1.0|100|endpoint|critical| desc=Endpoint file_access /I/fear.ppt by deannataylor No additional info', 'advanced_metadata': {'geo_location': 'Isle of Man', 'device_hash': 'd8595a4fb801a5ac7fcfe0987f50b16af71d52b2', 'user_agent': 'Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 5.01; Trident/5.0)', 'session_id': 'c1a4c54e-e3c9-459f-88af-afb52bd1f220', 'risk_score': 61.04, 'confidence': 0.33}, 'user': 'deannataylor', 'action': 'file_access', 'object': '/I/fear.ppt', 'process_id': 8141.0, 'parent_process': 'explorer.exe', 'additional_info': 'No additional info', 'description': 'Endpoint file_access /I/fear.ppt by deannataylor No additional info', 'behavioral_analytics': {'baseline_deviation': 1.84, 'entropy': 3.6, 'frequency_anomaly': False, 'sequ

In [80]:
dataset[0]["response"]

'random_forest - The event contains structured attributes (severity, action type, file path, user), making a supervised classification model like Random Forest ideal for endpoint activity detection and categorization.'

We notice all answers like about have a ####, so we extract it:

In [14]:
def extract_hash_answer(text):
    if "####" not in text: return None
    return text.split("####")[1].strip()
extract_hash_answer(dataset[0]["response"])

We now create a system prompt which can be customized. We add 4 extra symbols for working out or thinking / reasoning sections and a final answer:

In [81]:


system_prompt = \
f"""ill give you a log like this and you pick which ai model to use

cnn, random forest, isolation forest, naivebayes, bert, autoencoder, dqn, other

(PICK ONLY ONE) and a short one sentence reasoning

The format is <AI Model> - <Reasoning>
"""
system_prompt

'ill give you a log like this and you pick which ai model to use \n\ncnn, random forest, isolation forest, naivebayes, bert, autoencoder, dqn, other \n\n(PICK ONLY ONE) and a short one sentence reasoning\n\nThe format is <AI Model> - <Reasoning>\n'

Let's map the dataset! and see the first row:

In [76]:
system_prompt = str(system_prompt)

system_prompt

'ill give you a log like this and you pick which ai model to use \n\ncnn, random forest, isolation forest, naivebayes, bert, autoencoder, dqn, other \n\n(PICK ONLY ONE) and a short one sentence reasoning\n\nThe format is <AI Model> - <Reasoning>\n'

In [77]:
# from datasets import Features, Sequence, Value

# features = Features({
#     "prompt": Sequence({"role": Value("string"), "content": Value("string")}),
#     "answer": Value("string"),
# })

# dataset = dataset.map(
#     lambda x: {
#         "prompt": [
#             {"role": "system", "content": (system_prompt)},
#             {"role": "user",   "content": '"' + str(x.get("user", "")).replace('"', '\\"') + '"'},
#         ],
#         "answer": (x.get("response", "")),
#     },
#     features=features,
#     remove_columns=dataset.column_names,
# )

# dataset[0]


In [82]:
# dataset = dataset.map(lambda x: {
#     "prompt" : [
#         {"role": "system", "content": system_prompt},
#         {"role": "user",   "content": x["user"]},
#     ],
#     "answer": x["response"],
# })
# str(dataset[0])

#original
dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": str(x["user"])},
    ],
    "answer": (x["response"]),
})
dataset[0]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

{'uid': '8e785e09-5213-46b1-a6eb-b7e40998905b',
 'user': {'event_id': '8e785e09-5213-46b1-a6eb-b7e40998905b',
  'timestamp': datetime.datetime(2025, 5, 28, 23, 46, 49),
  'event_type': 'endpoint',
  'source': 'Microsoft Sentinel v1.0.0',
  'severity': 'critical',
  'raw_log': 'CEF:0|Microsoft Sentinel v1.0.0|SIEM|1.0|100|endpoint|critical| desc=Endpoint file_access /I/fear.ppt by deannataylor No additional info',
  'advanced_metadata': {'geo_location': 'Isle of Man',
   'device_hash': 'd8595a4fb801a5ac7fcfe0987f50b16af71d52b2',
   'user_agent': 'Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 5.01; Trident/5.0)',
   'session_id': 'c1a4c54e-e3c9-459f-88af-afb52bd1f220',
   'risk_score': 61.04,
   'confidence': 0.33},
  'user': 'deannataylor',
  'action': 'file_access',
  'object': '/I/fear.ppt',
  'process_id': 8141.0,
  'parent_process': 'explorer.exe',
  'additional_info': 'No additional info',
  'description': 'Endpoint file_access /I/fear.ppt by deannataylor No additional info',
  'be

We create a regex format to match the reasoning sections and answers:

In [83]:
import re

match_format = re.compile(
    rf"^[\s]{{0,}}"\
    rf"{reasoning_start}.+?{reasoning_end}.*?"\
    rf"{solution_start}(.+?){solution_end}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)

We verify it works:

In [62]:
match_format.search(
    "<start_working_out>Let me think!<end_working_out>"\
    "<SOLUTION>2</SOLUTION>",
)

<re.Match object; span=(0, 71), match='<start_working_out>Let me think!<end_working_out>>

We now want to create a reward function to match the format exactly - we reward it with 3 points if it succeeds:

In [63]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

If it fails, we want to reward the model if it at least follows the format partially, by counting each symbol:

In [64]:
def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Count how many keywords are seen - we penalize if too many!
        # If we see 1, then plus some points!
        score += 0.5 if response.count(reasoning_start) == 1 else -0.5
        score += 0.5 if response.count(reasoning_end)   == 1 else -0.5
        score += 0.5 if response.count(solution_start)  == 1 else -0.5
        score += 0.5 if response.count(solution_end)    == 1 else -0.5
        scores.append(score)
    return scores

In [85]:
import re

# Helper: extract whatever the model put as its final answer (text)
# 1) Prefer content between your tags:  <solution_start> ... <solution_end>
# 2) Otherwise fall back to a line like: "#### <answer>"
FINAL_LINE_RE = re.compile(r"(?m)^\s*####\s*(?P<ans>[^\n]+?)\s*$")
ONLY_NUMBER_RE = re.compile(r"^\s*[-+]?\d+(?:\.\d+)?\s*$")

def extract_solution_text(response: str) -> str:
    if solution_start in response and solution_end in response:
        try:
            return response.split(solution_start, 1)[1].split(solution_end, 1)[0].strip()
        except Exception:
            pass
    m = FINAL_LINE_RE.search(response)
    return m.group("ans").strip() if m else ""

def is_text_answer(ans: str) -> bool:
    # Accept if it has at least one letter and is not just a number
    return bool(ans) and re.search(r"[A-Za-z]", ans) and not ONLY_NUMBER_RE.match(ans)

def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        response = completion[0]["content"]
        ans = extract_solution_text(response)
        score = 3.0 if is_text_answer(ans) else 0.0
        scores.append(score)
    return scores

def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        response = completion[0]["content"]
        score = 0.0

        # keep your tag-count heuristics
        score += 0.5 if response.count(reasoning_start) == 1 else -0.5
        score += 0.5 if response.count(reasoning_end)   == 1 else -0.5
        score += 0.5 if response.count(solution_start)  == 1 else -0.5
        score += 0.5 if response.count(solution_end)    == 1 else -0.5

        # new: text-specific checks for the extracted final answer
        ans = extract_solution_text(response)
        if is_text_answer(ans):
            score += 0.5  # good: looks like text, not just a number
            n_words = len(ans.split())
            if 1 <= n_words <= 40:
                score += 0.5   # concise final text
            elif n_words > 120:
                score -= 0.5   # overly long for a "final answer"
        else:
            score -= 0.5       # missing/empty or purely numeric

        scores.append(score)
    return scores


Finally, we want to extract the generated answer, and reward or penalize it! We also reward it based on how close the answer is to the true one via ratios:

In [66]:
def check_answer(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_format.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(0)
            continue
        # Correct answer gets 3 points!
        if guess == true_answer:
            score += 3.0
        # Match if spaces are seen
        elif guess.strip() == true_answer.strip():
            score += 1.5
        else:
            # We also reward it if the answer is close via ratios!
            # Ie if the answer is within some range, reward it!
            try:
                ratio = float(guess) / float(true_answer)
                if   ratio >= 0.9 and ratio <= 1.1: score += 0.5
                elif ratio >= 0.8 and ratio <= 1.2: score += 0.25
                else: score -= 1.0 # Penalize wrong answers
            except:
                score -= 0.5 # Penalize
        scores.append(score)
    return scores

Also sometimes it might not be 1 number as the answer, but like a sentence for example "The solution is $20" -> we extract 20.

In [67]:
match_numbers = re.compile(
    rf"{solution_start}.*?([\d\.]{{1,}})",
    flags = re.MULTILINE | re.DOTALL
)
match_numbers.findall("<SOLUTION>  0.34  </SOLUTION>")

['0.34']

In [68]:
def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_numbers.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    print('*'*20, f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(0)
            continue
        # Convert to numbers
        try:
            true_answer = float(true_answer.strip())
            guess       = float(guess.strip())
            scores.append(1.5 if guess == true_answer else 0.0)
        except:
            scores.append(0)
            continue
    return scores

In [69]:
import re, difflib
from fractions import Fraction

# --- helpers ---
_OnlyNum = re.compile(r'^\s*[-+]?(\d+(?:[,_]\d{3})*|\d+)(?:\.\d+)?(?:[eE][-+]?\d+)?\s*%?\s*$')

def _normalize_text(s: str) -> str:
    s = s.strip().lower()
    # collapse whitespace
    s = re.sub(r'\s+', ' ', s)
    # drop most punctuation (keep % for possible semantics)
    s = re.sub(r'[^\w\s%]', '', s)  # remove punctuation except %; adjust if needed
    return s.strip()

def _text_similarity(a: str, b: str) -> float:
    # similarity on normalized strings
    a_n, b_n = _normalize_text(a), _normalize_text(b)
    return difflib.SequenceMatcher(None, a_n, b_n).ratio()

def _to_float(s: str):
    if s is None:
        return None
    t = s.strip()
    if not t:
        return None
    # percentage?
    is_pct = t.endswith('%')
    if is_pct:
        t = t[:-1]

    # fractions like "1/2"
    if '/' in t and not re.search(r'[A-Za-z]', t):
        try:
            val = float(Fraction(t.replace(' ', '')))
            return val / 100.0 if is_pct else val
        except Exception:
            pass

    # remove thousands separators/underscores/spaces
    t = t.replace(',', '').replace('_', '').replace(' ', '')

    try:
        val = float(t)
        return val / 100.0 if is_pct else val
    except Exception:
        return None

# --- TEXT ANSWER CHECKER ---
def check_answer(prompts, completions, answer, **kwargs):
    # Extract candidate answers using your existing regex `match_format`
    responses = [completion[0]["content"] for completion in completions]
    extracted = [
        m.group(1) if (m := match_format.search(r)) is not None else None
        for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted, answer):
        if guess is None:
            scores.append(0.0)
            continue

        # 1) exact match (verbatim)
        if guess == true_answer:
            scores.append(3.0)
            continue

        # 2) whitespace-only differences
        if guess.strip() == true_answer.strip():
            scores.append(2.0)
            continue

        # 3) case/punct/spacing-insensitive equality
        if _normalize_text(guess) == _normalize_text(true_answer):
            scores.append(1.5)
            continue

        # 4) fuzzy similarity on normalized text
        sim = _text_similarity(guess, true_answer)
        if   sim >= 0.90: scores.append(1.0)
        elif sim >= 0.80: scores.append(0.5)
        else:             scores.append(-0.5)  # penalize clear mismatch

    return scores

# --- NUMERIC ANSWER CHECKER (more robust parsing) ---
def check_numbers(prompts, completions, answer, **kwargs):
    # Extract numeric-looking substrings using your existing `match_numbers`
    responses = [completion[0]["content"] for completion in completions]
    extracted = [
        m.group(1) if (m := match_numbers.search(r)) is not None else None
        for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted, answer):
        if guess is None:
            scores.append(0.0)
            continue
        try:
            g = _to_float(guess)
            t = _to_float(true_answer if isinstance(true_answer, str) else str(true_answer))
            if g is None or t is None:
                scores.append(0.0)
                continue

            # exact numeric equality
            if g == t:
                scores.append(1.5)
                continue

            # relative tolerance buckets
            # avoid division by zero: use max(|t|, 1e-12)
            denom = max(abs(t), 1e-12)
            rel_err = abs(g - t) / denom

            if   rel_err <= 0.01: scores.append(1.0)   # within 1%
            elif rel_err <= 0.05: scores.append(0.75)  # within 5%
            elif rel_err <= 0.10: scores.append(0.5)   # within 10%
            else:                  scores.append(0.0)
        except Exception:
            scores.append(0.0)
    return scores


<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [70]:
max_prompt_length = 256

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 50,
    save_steps = 50,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [71]:
# 1) Rename your target to what TRL expects
# if "answer" in dataset.column_names:
#     dataset = dataset.rename_column("answer", "completion")
# if "response" in dataset.column_names and "completion" not in dataset.column_names:
#     dataset = dataset.rename_column("response", "completion")
if "label" in dataset.column_names:
    # either drop it OR keep it, but only if you ALSO have "completion"
    dataset = dataset.remove_columns(["label"])

# 2) Keep only the accepted keys for this mode: {"prompt","completion"}
# keep = {"prompt", "completion"}
# dataset = dataset.remove_columns([c for c in dataset.column_names if c not in keep])

# print(dataset.column_names)  # should be ['prompt', 'completion']


In [72]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_answer,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

Unsloth: Switching to float32 training since model cannot work with float16


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 6,522,880 of 1,006,408,832 (0.65% trained)
`generation_config` default values have been modified to match model-specific defaults: {'max_length': 32768, 'top_p': 0.95}. If this is not desired, please set these values explicitly.


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / match_format_exactly / mean,rewards / match_format_exactly / std,rewards / match_format_approximately / mean,rewards / match_format_approximately / std,rewards / check_answer / mean,rewards / check_answer / std
1,0.000000,-2.500000,0.000000,287.500000,154.000000,418.000000,0.000000,287.500000,154.000000,418.000000,0.000000,0.000000,0.000000,-2.500000,0.000000,0.000000,0.000000
2,0.000000,-2.500000,0.000000,33.250000,1.000000,47.000000,0.000000,33.250000,1.000000,47.000000,0.000000,0.000000,0.000000,-2.500000,0.000000,0.000000,0.000000
3,0.000000,-2.500000,0.000000,339.500000,100.000000,768.000000,0.250000,196.666672,100.000000,315.000000,0.000000,0.000000,0.000000,-2.500000,0.000000,0.000000,0.000000
4,0.000000,-2.500000,0.000000,8.000000,1.000000,29.000000,0.000000,8.000000,1.000000,29.000000,0.000000,0.000000,0.000000,-2.500000,0.000000,0.000000,0.000000
5,0.000000,-2.500000,0.000000,202.000000,1.000000,495.000000,0.000000,202.000000,1.000000,495.000000,0.000000,0.000000,0.000000,-2.500000,0.000000,0.000000,0.000000
6,0.000000,-2.500000,0.000000,139.500000,1.000000,215.000000,0.000000,139.500000,1.000000,215.000000,0.000000,0.000000,0.000000,-2.500000,0.000000,0.000000,0.000000
7,0.000000,-2.500000,0.000000,166.000000,2.000000,516.000000,0.000000,166.000000,2.000000,516.000000,0.000000,0.000000,0.000000,-2.500000,0.000000,0.000000,0.000000
8,0.000000,-2.500000,0.000000,159.250000,1.000000,620.000000,0.000000,159.250000,1.000000,620.000000,0.000000,0.000000,0.000000,-2.500000,0.000000,0.000000,0.000000
9,0.000000,-2.500000,0.000000,177.500000,40.000000,434.000000,0.000000,177.500000,40.000000,434.000000,0.000000,0.000000,0.000000,-2.500000,0.000000,0.000000,0.000000
10,0.000000,-2.500000,0.000000,140.750000,1.000000,375.000000,0.000000,140.750000,1.000000,375.000000,0.000000,0.000000,0.000000,-2.500000,0.000000,0.000000,0.000000


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / match_format_exactly / mean,rewards / match_format_exactly / std,rewards / match_format_approximately / mean,rewards / match_format_approximately / std,rewards / check_answer / mean,rewards / check_answer / std
1,0.000000,-2.500000,0.000000,287.500000,154.000000,418.000000,0.000000,287.500000,154.000000,418.000000,0.000000,0.000000,0.000000,-2.500000,0.000000,0.000000,0.000000
2,0.000000,-2.500000,0.000000,33.250000,1.000000,47.000000,0.000000,33.250000,1.000000,47.000000,0.000000,0.000000,0.000000,-2.500000,0.000000,0.000000,0.000000
3,0.000000,-2.500000,0.000000,339.500000,100.000000,768.000000,0.250000,196.666672,100.000000,315.000000,0.000000,0.000000,0.000000,-2.500000,0.000000,0.000000,0.000000
4,0.000000,-2.500000,0.000000,8.000000,1.000000,29.000000,0.000000,8.000000,1.000000,29.000000,0.000000,0.000000,0.000000,-2.500000,0.000000,0.000000,0.000000
5,0.000000,-2.500000,0.000000,202.000000,1.000000,495.000000,0.000000,202.000000,1.000000,495.000000,0.000000,0.000000,0.000000,-2.500000,0.000000,0.000000,0.000000
6,0.000000,-2.500000,0.000000,139.500000,1.000000,215.000000,0.000000,139.500000,1.000000,215.000000,0.000000,0.000000,0.000000,-2.500000,0.000000,0.000000,0.000000
7,0.000000,-2.500000,0.000000,166.000000,2.000000,516.000000,0.000000,166.000000,2.000000,516.000000,0.000000,0.000000,0.000000,-2.500000,0.000000,0.000000,0.000000
8,0.000000,-2.500000,0.000000,159.250000,1.000000,620.000000,0.000000,159.250000,1.000000,620.000000,0.000000,0.000000,0.000000,-2.500000,0.000000,0.000000,0.000000
9,0.000000,-2.500000,0.000000,177.500000,40.000000,434.000000,0.000000,177.500000,40.000000,434.000000,0.000000,0.000000,0.000000,-2.500000,0.000000,0.000000,0.000000
10,0.000000,-2.500000,0.000000,140.750000,1.000000,375.000000,0.000000,140.750000,1.000000,375.000000,0.000000,0.000000,0.000000,-2.500000,0.000000,0.000000,0.000000


TrainOutput(global_step=50, training_loss=0.0, metrics={'train_runtime': 2381.3176, 'train_samples_per_second': 0.084, 'train_steps_per_second': 0.021, 'total_flos': 0.0, 'train_loss': 0.0})

<a name="Inference"></a>
### Inference
Now let's try the model we just trained!

In [74]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": """
  "event_id": "3b00489f-5c83-4b94-a0cc-54e19c790edc",
  "timestamp": "2025-01-18T20:07:20",
  "severity": "info",
  "raw_log": "CEF:0|Carbon Black v7.8.0|SIEM|1.0|100|firewall|info| desc=Firewall drop SSH traffic from 100.11.91.183:51767 to 35.35.93.73:580 No additional info",
  "user": null,
  "action": "drop",
  "description": "Firewall drop SSH traffic from 100.11.91.183:51767 to 35.35.93.73:580 No additional info",
  "src_ip": "100.11.91.183",
  "dst_ip": "35.35.93.73",
  "category": null
"""},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<SOLUTION>
The log indicates that a firewall dropped SSH traffic. The source IP is 100.11.91.183 and the destination is 35.35.93.73. The action is a "drop" indicating that the traffic was blocked.



<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("gemma-3")  # Local saving
tokenizer.save_pretrained("gemma-3")
# model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving

('gemma-3/tokenizer_config.json',
 'gemma-3/special_tokens_map.json',
 'gemma-3/tokenizer.model',
 'gemma-3/added_tokens.json',
 'gemma-3/tokenizer.json')

### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3-finetune`. Set `if False` to `if True` to let it run!

In [ ]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3-finetune", tokenizer,
        token = "hf_..."
    )

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune",
        tokenizer,
        quantization_method = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3-finetune",
        tokenizer,
        quantization_method = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
        token = "hf_...",
    )

Now, use the `gemma-3-finetune.gguf` file or `gemma-3-finetune-Q4_K_M.gguf` file in llama.cpp.

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>

  This notebook and all Unsloth notebooks are licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).
